# LangChain Agents Tutorial

This notebook demonstrates how to create and use agents in LangChain. We'll build a ReAct agent that can search the web and get weather information.

## Step 1: Setting up the OpenAI API Key

First, we need to set up our OpenAI API key to use the language model.


In [ ]:
import os
os.environ["GROQ_API_KEY"] = "[YOUR_GROQ_API_KEY]"

## Step 2: Installing Required Packages

We need to install the necessary LangChain packages and other dependencies for our agent.


In [2]:
!pip install -q langchain-groq langchain-community langchain-core requests duckduckgo-search ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## Step 3: Importing Core Libraries

Let's import the essential LangChain components and other libraries we'll need.


In [3]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
import requests

## Step 4: Setting up the Search Tool

We'll use DuckDuckGo search as one of our tools to allow the agent to search for information on the web.


In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

## Step 5: Creating a Custom Weather Tool

Here we define a custom tool that can fetch weather data for any city using the WeatherStack API. This tool will be available to our agent.


In [ ]:
@tool
def get_weather_data(city: str) -> str:
  """
  This tool fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=[YOUR_WEATHERSTACK_API_KEY]&query={city}' # Enter the weatherstack api key before use

  response = requests.get(url)

  return response.json()

## Step 6: Initializing the Language Model

We create an instance of the ChatOpenAI model that will power our agent's reasoning capabilities.


In [6]:
# llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0)
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    # Enable tool calling for this model
    model_kwargs={"tool_choice": "auto"}
)

## Step 7: Importing Agent Components

We import the necessary components to create a ReAct agent and agent executor.


In [7]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

## Step 8: Loading the ReAct Prompt

We pull the standard ReAct (Reasoning and Acting) prompt from LangChain Hub. This prompt template guides the agent on how to reason through problems and take actions.


In [8]:
# Step 2: Pull the ReAct prompt from LangChain Hub
base_prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt

In [ ]:
# Add stopping condition to prevent infinite loop

stopping_instructions = """
You will reason step-by-step using the format:
Thought: <your reasoning>
Action: <tool name>
Action Input: <input to the tool>

**Stopping condition:**
If you have gathered all required information to answer the user's question,
STOP calling more tools.
Instead, print your answer starting with 'Final Answer:' on a new line.
Do NOT print any more Thoughts or Actions after the final answer.

Example:
Thought: I should find the capital of Maharashtra.
Action: search_tool
Action Input: "capital of Maharashtra"

Thought: I found the info I need.
Final Answer: The capital of Maharashtra is Mumbai.
"""

In [ ]:
modified_prompt = base_prompt + "\n" + stopping_instructions # add stopping condition to the predefined prompt

## Step 9: Creating the ReAct Agent

Now we create our ReAct agent by combining the language model, our tools (search and weather), and the ReAct prompt template.


In [11]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=modified_prompt
)

## Step 10: Creating the Agent Executor

The AgentExecutor wraps our agent and handles the execution loop, managing tool calls and responses.


In [12]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True,
    handle_parsing_errors=True
)

## Step 11: Testing the Agent

Let's test our agent with a complex query that requires both searching for information and getting weather data. The agent will need to:
1. Find the capital of Madhya Pradesh
2. Get the current weather for that city


In [17]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
# response = agent_executor.invoke({"input": "Find the weather condition of Bhopal."})
print(response)



> Entering new AgentExecutor chain...
Question: Find the capital of Madhya Pradesh, then find its current weather condition  
Thought: I need to determine the capital city of Madhya Pradesh.  
Action: duckduckgo_search  
Action Input: "capital of Madhya Pradesh"  Less-fertile red-to-yellow soils are spread over much of eastern Madhya Pradesh . ... Madhya Pradesh has a number of national parks and many wildlife ... Bhopal is the capital city of Madhya Pradesh . ... Madhya Pradesh has a population of 8.50 crore (as of 2020), out of which 90.9%Â are Hindus ... Madhya Pradesh state is made up of 48 districts, which are grouped into nine divisions: Bhopal , Chambal, Gwalior , Hoshangabad, Indore , Jabalpur ... Madhya Pradesh was created in 1950 from the former British Central Provinces and Berar and the princely states of Makrai and Chhattisgarh, with Nagpur ... The capital of Madhya Pradesh is Bhopal. ... A large section of the Madhya Pradesh population belongs to the category known as S

In [18]:
response['output']

'The capital of Madhya Pradesh is **Bhopal**.  \nAccording to the latest weather data, Bhopal is experiencing **haze** with a temperature of **25\u202f°C**, humidity of **79\u202f%**, light winds from the east (about 6\u202fkm/h), and a pressure of **1013\u202fhPa**. The observation time is 03:55\u202fPM local time, and it is currently night (no daylight).'

## Summary

This notebook demonstrated how to:
- Set up LangChain agents with custom tools
- Use the ReAct framework for reasoning and acting
- Combine multiple tools (search and weather API) in a single agent
- Execute complex multi-step queries that require both information retrieval and data processing

The agent successfully found that Bhopal is the capital of Madhya Pradesh and retrieved its current weather conditions by using both the search tool and the weather API tool in sequence.
